## Identify matches between the SWS/CASSIS datasets.

In [17]:
import astropy
import matplotlib
import numpy as np
import pandas as pd
import swsnet

from astropy.coordinates import SkyCoord
from concurrent.futures import ProcessPoolExecutor

In [33]:
def get_ang_sep(ra1, dec1, ra2, dec2):
    """
    Calculate the angular distance between two coordinates.

    Parameters
    ----------
    ra1 : float, array
        Right ascension of the first object in degrees.
    dec1 : float, array
        Declination of the first object in degrees.
    ra2 : float, array
        Right ascension of the second object in degrees.
    dec2 : float, array
        Declination of the second object in degrees.
    Returns
    -------
    Angle : float, array
        The angular distance in DEGREES between the first
        and second coordinate in the sky.

    """
    
    delt_lon = (ra1 - ra2)*np.pi/180.
    delt_lat = (dec1 - dec2)*np.pi/180.
    # Haversine formula
    dist = 2.0*np.arcsin( np.sqrt( np.sin(delt_lat/2.0)**2 + \
         np.cos(dec1*np.pi/180.)*np.cos(dec2*np.pi/180.)*np.sin(delt_lon/2.0)**2 ) )  

    ang_sep = dist/np.pi*180.
     
    return ang_sep

In [40]:
def get_matches(in_tuple):

    global czip
    global c_aor
    global s_tdt
    
    sindex, sra, sdec = in_tuple
    match_list = []
    
    for index, value in enumerate(czip):
        cindex, cra, cdec = value
        ang_sep = get_ang_sep(sra, sdec, cra, cdec)
        if ang_sep*3600 <= 30:
            match_tuple = (c_aor[cindex], s_tdt[sindex], ang_sep*3600, cindex, sindex)
            match_list.append(match_tuple)
    
    if match_list:
        return match_list
    else:
        return None

In [41]:
cass = pd.read_pickle('cassis/metadata.pkl')
sws = pd.read_pickle('isosws_atlas/metadata.pkl')

cra = cass.ra.values
cdec = cass.dec.values

sra = sws.ra.values
sdec = sws.dec.values

c_aor = cass.aorkey.values
s_tdt = sws.tdt.values

Array containing all possible combinations:

In [42]:
czip = []
szip = []

global czip
global c_aor
global s_tdt

for cindex, cvalue in enumerate(cra):
    czip.append((cindex, cra[cindex], cdec[cindex]))
    
for sindex, svalue in enumerate(sra):
    szip.append((sindex, sra[sindex], sdec[sindex]))    

In [43]:
len(czip), len(szip)

(6732, 1235)

### Use parallelization to calculate.

In [44]:
pool = ProcessPoolExecutor(max_workers=14)
results = list(pool.map(get_matches, szip))

In [46]:
results = [x for x in results if x is not None]
results

[[('21787648', '28501420', 13.801969580651912, 3428, 5)],
 [('21788416', '23200502', 9.683969511351991, 3431, 25)],
 [('10440704', '42500402', 12.890780337495183, 74, 29)],
 [('9069568', '24701422', 8.963445194714112, 6598, 34)],
 [('17505792', '68600302', 10.091857207255265, 1830, 49)],
 [('22268416', '77700106', 9.173995006160268, 4018, 50)],
 [('22268416', '24800901', 9.151258589598555, 4018, 51)],
 [('22268416', '59502003', 9.151258589598555, 4018, 52)],
 [('22268416', '64500309', 9.151258589598555, 4018, 53)],
 [('22268416', '83701909', 9.151258589598555, 4018, 54)],
 [('22268416', '28301904', 8.978172072375347, 4018, 55)],
 [('22268416', '65400106', 9.396940950677656, 4018, 56)],
 [('10661632', '17101710', 4.494310002074252, 113, 57)],
 [('10661632', '17101809', 4.494310002074252, 113, 58)],
 [('18505984', '17902439', 9.104975744635418, 2228, 70),
  ('28720896', '17902439', 9.786276430070545, 5743, 70)],
 [('9070848', '63300301', 7.924278383248893, 6602, 84)],
 [('19150848', '115

In [47]:
len(results)

133